In [ ]:
from azureml.core import Workspace, Dataset, Experiment, Run
from azureml.core.compute import ComputeTarget,AmlCompute
from azureml.core.model import Model, InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.compute_target import ComputeTargetException
from sklearn.model_selection import train_test_split
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
import logging, requests,json
import urllib.request

# Get current workspace info
ws = Workspace.from_config()

# Since the dataset is already uploaded and listed in the ws
# retrieve the dataset by name
dataset = Dataset.get_by_name(ws, name='wine')

# For easier manipulation, let's transform the tabular dataset into
# a pandas dataframe
df = dataset.to_pandas_dataframe()

In [ ]:
# Splitting the dataset in train and test for further use
train_data, test_data = train_test_split(df, test_size=0.1, random_state=42)
# Defining the label
label = "quality"

# Since AutoML supports only Tabular Dataset, the train dataset needs to be converted
# in the correct format. 
# First let's retrieve the datastore 
datastore = ws.get_default_datastore()
# And use the sdk to register and convert from pandas to Tabular 
train_dataset = Dataset.Tabular.register_pandas_dataframe(train_data,datastore,'wine_train_dataset',description='Train dataset for wine')
#test_dataset = Dataset.Tabular.register_pandas_dataframe(test_data,datastore,'wine_test_dataset',description='Test dataset for wine')


In [ ]:
# AutoML need a compute target to run
# You can use an already existing one or 
# you can create a new one

compute_name = 'your_compute_instance'

try:
    compute_target = ComputeTarget(workspace=ws,name=compute_name)
    print('Found existing compute resource, using it')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(
        vm_size= 'STANDARD_D12_V2', #you can change the size of the vm
        max_nodes=2 #you can change the max number of nodes
    )
    compute_target = ComputeTarget.create(ws, compute_name, compute_config)

compute_target.wait_for_completion(show_output=True)

In [ ]:
# Automl config where you define the taks type,
# the primary metric, the training dataset,
# the label, the compute target.   

automl_config = AutoMLConfig(
                                 task='classification',
                                 debug_log ='automl_notebook_errors.log',
                                 primary_metric='AUC_weighted',
                                 experiment_timeout_minutes=30,
                                 training_data=train_dataset,
                                 label_column_name=label,
                                 compute_target = compute_target,
                                 n_cross_validations=5,
                                 verbosity=logging.INFO,
                                 featurization='auto',
                                 enable_voting_ensemble=False,
                                 enable_stack_ensemble=False
                                )

In [ ]:
# To run AutoML an experiment is required
experiment_name = 'AutoML-Notebook'
experiment = Experiment(ws, experiment_name)

# Now with the experiment create
# we can submit the AutoML and it creates a run
run = experiment.submit(automl_config, show_output=True)

# The output of the run is the best model
best_run, fitted_model = run.get_output()
print(fitted_model.steps)

In [ ]:
# We can use the test dataset to check the performance 
# of the best model
y_test = test_data.pop("quality")

y_classify = fitted_model.predict(test_data)

print("Predicted:  " + y_classify[:10] + " Value: " + y_test[:10])


In [ ]:
# Register the best model into Models
model_name = best_run.properties['model_name']
description = 'AutoML jupyter example'
tags = None

model = run.register_model(model_name = model_name, 
                                  description = description, 
                                  tags = tags)

In [ ]:
# AutoML auto-generates the score.py file
# which it's useful when we want to deploy the model
# as a web service. We can download it and use it as an
# entry point for the API

script_file_name = 'score.py'
file_dwn = best_run.download_file('./outputs/scoring_file_v_1_0_0.py', script_file_name)

print(file_dwn)

In [ ]:
# Set the model configurations: environment and entry script
inference_config = InferenceConfig(environment = best_run.get_environment(), 
                                   entry_script = './score.py')

# Set the container configurations
aciconfig = AciWebservice.deploy_configuration(cpu_cores = 2, 
                                               memory_gb = 4, 
                                               tags = {'type': "automl-jupyter"},
                                               description = "Automl jupyter sample service")

aci_service_name = 'automl-jupyter-model'
print(aci_service_name)

# Deploy the model into the ACI
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

In [ ]:
# Test the container deployed with a data example.

scoring_uri = aci_service.scoring_uri

# Request data goes here
data = {"data": [
    {
      "fixed acidity": 8.8,
      "volatile acidity": 0.32,
      "citric acid": 0.57,
      "residual sugar": 2.3,
      "chlorides": 0.069,
      "free sulfur dioxide": 29,
      "total sulfur dioxide": 37,
      "density": 0.997,
      "pH": 3.15,
      "sulphates": 0.61,
      "alcohol": 9.6
    }
  ],
  "method": "predict"
  }

input_data = json.dumps(data)

# Set the content type
headers = {'Content-Type': 'application/json'}

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)

print(resp.status_code)
print("The wine submitted is " + resp.json())